In [1]:
%pip install mecode

import mecode
from mecode import GMatrix

Note: you may need to restart the kernel to use updated packages.


# Helpers

In [2]:
def pen_down(g: GMatrix) -> None:
    """Lower the pen.

    Args:
        g (GMatrix): GMatrix instance to use for GCode generation.
    """

    g.move(z=-3.3)
    g.write("M9")


def pen_up(g: GMatrix) -> None:
    """Raise the pen.

    Args:
        g (GMatrix): GMatrix instance to use for GCode generation.
    """

    g.move(z=3.3)
    g.write("M8")


def get_gmatrix(outfile: str = None) -> GMatrix:
    """Create a GMatrix instance and write the starting GCode.

    Args:
        outfile (str, optional): Path to the output file. Defaults to None.

    Returns:
        GMatrix: GMatrix instance.
    """

    start_code = "G91 ;relative\nG21 ;mm\nG92 X0 Y0 Z0 ;reset origin\nF2000;motion speed"
    g = GMatrix(outfile=outfile)
    g.push_matrix()
    g.write(start_code)
    return g


# Draw a unit

In [3]:
def draw_unit(g: GMatrix, path: list[list[tuple[float, float]]]) -> None:
    """Draw a unit of the repeating pattern. This method assumes the pen is raised at the starting position when called.

    Args:
        g (GMatrix): GMatrix instance to use for GCode generation.
        path (list[list[tuple[float, float]]]): List of segments to draw. Each segment is a list of points. The coordinates are relative to the current position.
    """

    for segment in path:
        # Convert relative coordinates to absolute coordinates
        segment = [(x + g.current_position['x'], y + g.current_position['y'])
                   for x, y in segment]

        # Move to the first point of the segment and start drawing
        x0, y0 = segment[0]
        g.abs_move(x0, y0)
        pen_down(g)

        # Draw the segment by moving to each point
        for x, y in segment[1:]:
            g.abs_move(x, y)

        # Finish drawing the segment. Raise the pen to prepare for the next segment.
        pen_up(g)


In [4]:
# Unit test for draw_unit
g: GMatrix = get_gmatrix()
pen_up(g)

square_pattern = [
        [(0, 10), (0, 0), (10, 0)] # The left and top sides of the square
    ] # This pattern contains only one segment.
draw_unit(g, square_pattern)

G91 ;relative
G91 ;relative
G21 ;mm
G92 X0 Y0 Z0 ;reset origin
F2000;motion speed
G1 X0.000000 Y0.000000 Z3.300000
M8
G90 ;absolute
G1 X0.000000 Y10.000000 Z3.300000
G91 ;relative
G1 X0.000000 Y0.000000 Z-3.300000
M9
G90 ;absolute
G1 X0.000000 Y0.000000 Z0.000000
G91 ;relative
G90 ;absolute
G1 X10.000000 Y0.000000 Z0.000000
G91 ;relative
G1 X0.000000 Y0.000000 Z3.300000
M8


# Draw repeating pattern

In [5]:
def draw_reapeating_pattern(
        g: GMatrix,
        width: float,
        height: float,
        path: list[list[tuple[float, float]]],
        scale: float,
        repeat: tuple[int, int]
        ) -> None:
    """Draw a unit of the repeating pattern. This method assumes the pen is raised at the starting position when called.

    Args:
        g (GMatrix): GMatrix instance to use for GCode generation.
        width (float): Width of the pattern in mm.
        height (float): Height of the pattern in mm.
        path (list[list[tuple[float, float]]]): List of segments to draw. Each segment is a list of points.
        scale (float): Scale factor to apply to the pattern.
        repeat (tuple[int, int]): Number of times to repeat the pattern in the x and y directions, respectively.
    """

    # Check if there is any point in path that is outside the limit of width and height
    for segment in path:
        for x, y in segment:
            if x > width or y > height:
                raise ValueError("Path is outside the dimension limit")

    # Scale the pattern
    path = [[(x * scale, y * scale) for x, y in segment] for segment in path]
    width = width * scale
    height = height * scale

    # Check if the repeating pattern takes more space than the page
    # Page limit: 205x275 mm
    if width * repeat[0] > 205 or height * repeat[1] > 275:
        raise ValueError("Repeating pattern is outside the dimension limit")

    # Draw the pattern from the current position
    x_initial = g.current_position['x']
    y_initial = g.current_position['y']
    x = x_initial
    y = y_initial
    for y_rep in range(repeat[1]):

        # Draw each row of the pattern
        for x_rep in range(repeat[0]):
            draw_unit(g, path)
            x += width
            g.abs_move(x)
        
        # Move to the next row
        x = x_initial
        y += height
        g.abs_move(x_initial, y)


In [6]:
# Useful GCode snippets
end_code = ""

# Initialise GCode generation and write start code
g: GMatrix = get_gmatrix("output.gcode")

# Move the pen to a reasonable starting position
pen_up(g)
g.move(30, 30)

# Draw the pattern
square_pattern = [
        [(0, 10), (0, 0), (10, 0)] # The left and top sides of the square
    ] # This pattern contains only one segment.
draw_reapeating_pattern(g, 10, 10, square_pattern, 1, (4, 10))

# Write end code
g.pop_matrix()
g.write(end_code)
